In [218]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [219]:
def convert_columns_to_str(df,number_columns):
    """
    Функция для конвертации указанных столбцов в строковый тип и очистки от пробельных символов в начале и конце
    """
    
    for column in number_columns: # Перебираем список нужных колонок
        try:            
            df.iloc[:,column] = df.iloc[:,column].astype(str)
            # Очищаем колонку от пробельных символов с начала и конца
            df.iloc[:,column] = df.iloc[:,column].apply(lambda x: x.strip())
        except IndexError:
            print('Проверьте порядковые номера колонок которые вы хотите обработать.')
        

def convert_params_columns_to_int(lst):
    """
    Функция для конвератации значений колонок которые нужно обработать.
    Очищает от пустых строк, чтобы в итоге остался список из чисел в формате int
    """
    out_lst = [] # Создаем список в который будем добавлять только числа
    for value in lst: # Перебираем список
        try:
            # Обрабатываем случай с нулем, для того чтобы после приведения к питоновскому отсчету от нуля не получилась колонка с номером -1
            number = int(value)
            if number != 0:
                out_lst.append(value) # Если конвертирования прошло без ошибок то добавляем
            else:
                continue
        except: # Иначе пропускаем
            continue
    return out_lst
                


In [220]:
file_params = 'params.xlsx'
path_to_end_folder = 'data/'

In [221]:
first_df = pd.read_excel('data/Большой список.xlsx',keep_default_na=False)
second_df = pd.read_excel('data/Малый список.xlsx',keep_default_na=False)
# Считываем файл параметров без заголовка, чем проще тем лучше, пропущенные значения обозначим пустыми строками
params = pd.read_excel(file_params,header=None,keep_default_na=False)

In [222]:
params

,0,1
0,0,1
1,,2
2,,3
3,,
4,,
5,,
6,,
7,,
8,6,


In [223]:
# Преврашаем каждую колонку в список
params_first_columns = params[0].tolist()
params_second_columns = params[1].tolist()



In [224]:
params_second_columns

[1, 2, 3, '', '', '', '', '', '']

In [225]:
# Конвертируем в инт заодно проверяя корректность введенных данных
int_params_first_columns = convert_params_columns_to_int(params_first_columns)
int_params_second_columns = convert_params_columns_to_int(params_second_columns)

    


In [226]:
int_params_first_columns

[6]

In [227]:
int_params_second_columns

[1, 2, 3]

In [228]:
# Отнимаем 1 от каждого значения чтобы привести к питоновским индексам
int_params_first_columns = list(map(lambda x:x-1,int_params_first_columns))
int_params_second_columns = list(map(lambda x:x-1,int_params_second_columns))

In [229]:
int_params_second_columns

[0, 1, 2]

In [230]:
# Конвертируем нужные нам колонки в str
convert_columns_to_str(first_df,int_params_first_columns)
convert_columns_to_str(second_df,int_params_second_columns)

In [231]:
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
first_df['ID'] = first_df.iloc[:,int_params_first_columns].sum(axis=1)
second_df['ID'] = second_df.iloc[:,int_params_second_columns].sum(axis=1)


In [232]:
first_df['ID']

0        
1        
2        
3        
4        
       ..
998      
999      
1000     
1001     
1002     
Name: ID, Length: 1003, dtype: object

In [233]:
# очищаем от пробелов между словами
first_df['ID'] =first_df['ID'].apply(lambda x:x.replace(' ',''))
second_df['ID'] =second_df['ID'].apply(lambda x:x.replace(' ',''))

In [234]:
first_df['ID']

0        
1        
2        
3        
4        
       ..
998      
999      
1000     
1001     
1002     
Name: ID, Length: 1003, dtype: object

In [235]:
second_df['ID']

0           АлехинДанилаПрокопьевич
1          ЭсауловаВалерияПрокловна
2              ПанинГавриилВласович
3      ГрибановКонстантинСтепанович
4          МандрыкоТрофимЛеонидович
                   ...             
436      НиконоваАнтонинаМаксимовна
437       ФурмановАркадийЕлизарович
438        КорниецЛюдмилаМаксимовна
439       СеребровГавриилФеодосивич
440       СеребровГавриилФеодосивич
Name: ID, Length: 441, dtype: object

In [236]:
# Обрабатываем дубликаты

duplicates_first_df = first_df[first_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм


first_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма

duplicates_second_df = second_df[second_df.duplicated(subset=['ID'],keep=False)] # Сохраняем все значения у которых есть дубликаты в отдельный датафрейм
second_df.drop_duplicates(subset=['ID'],keep=False,inplace=True)# Удаляем дубликаты из датафрейма


In [237]:
duplicates_first_df

,Фамилия,Имя,Отчество,ФИО,Дата рождения,Номер телефона,Электронная почта,Адрес прописки,Страна,Регион,...,ИНН ФЗ/ИП,СНИЛС,ОМС,Специальность,Направление,Учебное заведение,Серия/Номер диплома,Регистрационный номер,Дата окончания обучения,ID
0,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,,,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,Сахалинская область,...,158493217864,,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,
1,Алехин,Данила,Прокопьевич,Алехин Данила Прокопьевич,14.04.1973,,,"Россия, г. Иркутск, Кирова ул., д. 1 кв.212",Россия,Сахалинская область,...,158493217864,,5950614844448865,Администратор базы данных,02.03.03 Математическое обеспечение и админист...,Санкт-Петербургский государственный университет,714531 1469061,11-701-701,2019,
2,Эсаулова,Валерия,Прокловна,Эсаулова Валерия Прокловна,20.08.1976,,,"Россия, г. Хабаровск, Дружная ул., д. 24 кв.29",Россия,Чувашская Республика,...,659323877844,,5993766290135682,Сурдолог,30.05.01 Медицинская биохимия,Московский государственный университет имени М...,221020 6258508,26-806-481,2007,
3,Панин,Гавриил,Власович,Панин Гавриил Власович,25.10.1990,,,"Россия, г. Дзержинск, Тихая ул., д. 18 кв.204",Россия,Ямало-Ненецкий АО,...,763641729189,,2559231278952211,Работник органов ЗАГСа,40.03.01 Юриспруденция,Казанский федеральный университет,604766 9441279,93-143-148,2018,
4,Грибанов,Константин,Степанович,Грибанов Константин Степанович,06.09.1989,,,"Россия, г. Махачкала, Белорусская ул., д. 25 к...",Россия,Самарская область,...,681365142211,,4713493566494435,Кинолог,40.05.02 Правоохранительная деятельность,Московский государственный университет имени М...,987039 5196117,54-333-851,2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Никонова,Антонина,Максимовна,Никонова Антонина Максимовна,03.01.1961,,,"Россия, г. Керчь, Тихая ул., д. 3 кв.135",Россия,Свердловская область,...,391391260298,,3296742751728743,Расточник,07.03.01 Архитектура,Петербургский государственный университет путе...,701920 2789029,23-897-389,2015,
999,Фурманов,Аркадий,Елизарович,Фурманов Аркадий Елизарович,17.09.1984,,,"Россия, г. Раменское, Садовый пер., д. 4 кв.14",Россия,Республика Татарстан,...,934401112263,,2399010395165533,Кардиолог,30.05.03 Медицинская кибернетика,Курский государственный медицинский университет,127947 2883424,90-544-900,2022,
1000,Корниец,Людмила,Максимовна,Корниец Людмила Максимовна,04.07.1974,,,"Россия, г. Киров, Советская ул., д. 14 кв.153",Россия,Курганская область,...,584017238395,,3407042725943029,Радиомеханик,"08.05.02 Строительство, эксплуатация, восстано...",Южно-Уральский государственный университет,451714 9128673,26-410-550,2011,
1001,Серебров,Гавриил,Феодосивич,Серебров Гавриил Феодосивич,27.05.1975,,,"Россия, г. Саранск, Железнодорожная ул., д. 1 ...",Россия,Московская область,...,993837704539,,7524798525031657,Биоинженер,47.03.01 Философия,Московский государственный университет имени М...,502723 5837429,91-004-258,2009,


In [238]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Данные которых нет во 2 таблице'
wb.create_sheet(title='Данные которых нет во 1 таблице', index=1)
wb.create_sheet(title='Совпадающие данные', index=2)
# Создаем листы для дубликатов
wb.create_sheet(title='Дубликаты первая таблица', index=3)
wb.create_sheet(title='Дубликаты вторая таблица', index=4)

# Создаем датафрейм
itog_df = pd.merge(first_df, second_df, how='outer', left_on=['ID'], right_on=['ID'],
                   indicator=True)

# Записываем каждый датафрейм в соответсвующий лист
left_df = itog_df[itog_df['_merge'] == 'left_only']


left_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Данные которых нет во 2 таблице'].append(r)

right_df = itog_df[itog_df['_merge'] == 'right_only']
right_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Данные которых нет во 1 таблице'].append(r)

both_df = itog_df[itog_df['_merge'] == 'both']
both_df.drop(['_merge'], axis=1, inplace=True)
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Совпадающие данные'].append(r)

# Записываем дубликаты в соответствующие листы
for r in dataframe_to_rows(duplicates_first_df, index=False, header=True):
    wb['Дубликаты первая таблица'].append(r)

for r in dataframe_to_rows(duplicates_second_df, index=False, header=True):
    wb['Дубликаты вторая таблица'].append(r)

In [239]:
# Сохраняем
t = time.localtime()
current_time = time.strftime('%H_%M_%S %d.%m.%Y', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder}/Совпадающие,уникальные данные  от {current_time}.xlsx')